In [1]:
import numpy as np
import pandas as pd

from astropy.io import fits

import auxil_ML as aux


In [184]:
cat = '3FGL'
if cat == '4FGL':
    #cat_name = '4FGL'
    cat_name = '4FGL-DR2'
else:
    cat_name = cat
    
nclasses = 3

folder = '../SOM'
#folder = '../ML_3FGL_paper_extra/data/python files/catas'
cat_fn = '%s/%s_prob_catalog_%iclasses.csv' % (folder, cat_name, nclasses)
cat_df = pd.read_csv(cat_fn, index_col=0)
labels = ['AGN', 'PSR', 'MIXED']
if nclasses == 3:
    labels = ['AGN', 'PSR', 'OTHER', 'MIXED']
res_dict = {}
res_dict['uncorrected'] = {}
res_dict['corrected'] = {}

# unweighted algorithms
algs_u = ['NN', 'LR', 'RF', 'BDT']
# oversampled algorithms
algs_o = ['NN_O', 'LR_O', 'RF_O', 'BDT_O']

algs = algs_u + algs_o


In [185]:
if cat_name == '4FGL-DR2':
    fn = 'data/gll_psc_v27.fit'
elif cat_name == '4FGL':
    fn = 'data/gll_psc_v22.fit'
elif cat_name == '3FGL':
    fn = '../PS_comparison/gll_psc_v16.fit'
    
with fits.open(fn) as data:
    data[1].header
    cat0_df = aux.hdu2df(data[1], index_name='Source_Name')

print(len(cat0_df), len(cat_df))
names0 = list(cat0_df.index)
for name in cat_df.index:
    names0.remove(name)
print(len(names0))

3034 3034
0


In [186]:
cat0_df.columns

Index(['RAJ2000', 'DEJ2000', 'GLON', 'GLAT', 'Conf_68_SemiMajor',
       'Conf_68_SemiMinor', 'Conf_68_PosAng', 'Conf_95_SemiMajor',
       'Conf_95_SemiMinor', 'Conf_95_PosAng', 'ROI_num', 'Signif_Avg',
       'Pivot_Energy', 'Flux_Density', 'Unc_Flux_Density', 'Flux1000',
       'Unc_Flux1000', 'Energy_Flux100', 'Unc_Energy_Flux100', 'Signif_Curve',
       'SpectrumType', 'Spectral_Index', 'Unc_Spectral_Index', 'beta',
       'Unc_beta', 'Cutoff', 'Unc_Cutoff', 'Exp_Index', 'Unc_Exp_Index',
       'PowerLaw_Index', 'Flux30_100', 'Unc_Flux30_100', 'nuFnu30_100',
       'Sqrt_TS30_100', 'Flux100_300', 'nuFnu100_300', 'Sqrt_TS100_300',
       'Flux300_1000', 'nuFnu300_1000', 'Sqrt_TS300_1000', 'Flux1000_3000',
       'nuFnu1000_3000', 'Sqrt_TS1000_3000', 'Flux3000_10000',
       'nuFnu3000_10000', 'Sqrt_TS3000_10000', 'Flux10000_100000',
       'nuFnu10000_100000', 'Sqrt_TS10000_100000', 'Variability_Index',
       'Signif_Peak', 'Flux_Peak', 'Unc_Flux_Peak', 'Time_Peak',
       'Peak_I

In [187]:
if 0:
    fn = '../ML_3FGL_paper_extra/data/python files/catas/%s_missing_values_src_test.csv' % cat_name
    print(fn)
    miss_df.to_csv(fn, index_label='%s_Source_Name' % cat)

In [188]:
unass_mask = cat_df['Category_%s' % cat] == 'UNAS'
ass_mask = cat_df['Category_%s' % cat] != 'UNAS'
other_mask = cat_df['Category_%s' % cat] == 'OTHER'
n_unas = np.sum(unass_mask)
n_as = np.sum(ass_mask)
print(n_as, n_unas)

2024 1010


In [189]:
set(cat_df['Category_%s' % cat])

{'AGN', 'OTHER', 'PSR', 'UNAS'}

In [192]:
print('%s number of pulsars: %i' % (cat, np.sum(cat_df['Category_%s' % cat] == 'PSR')))
print('%s number of AGNs: %i' % (cat, np.sum(cat_df['Category_%s' % cat] == 'AGN')))

3FGL number of pulsars: 167
3FGL number of AGNs: 1744


In [175]:
for label in labels:
    res_dict['uncorrected'][label] = np.sum((cat_df['Category_Prob'] == label) & unass_mask)
    print('%s: %i' % (label, res_dict['uncorrected'][label]))
print('Total:', n_unas)

AGN: 739
PSR: 64
OTHER: 274
MIXED: 590
Total: 1667


In [176]:
# outliers
for src in ['AGN', 'PSR']:
    total = 1. * sum((cat_df['Category_%s' % cat] == src))
    nmixed = sum((cat_df['Category_%s' % cat] == src) & (cat_df['Category_Prob'] == 'MIXED'))
    nwrong = sum((cat_df['Category_%s' % cat] == src) & (cat_df['Category_Prob'] != src)) - nmixed
    print('%s %s (%i total)' %(cat, src, total))
    print('Mixed: %i (%.3f)' % (nmixed, nmixed/total))
    print('Wrong: %i (%.3f)' % (nwrong, nwrong/total))

4FGL AGN (3503 total)
Mixed: 176 (0.050)
Wrong: 20 (0.006)
4FGL PSR (271 total)
Mixed: 77 (0.284)
Wrong: 27 (0.100)


In [177]:
sum((cat_df['Category_%s' % cat] == 'PSR') & (cat_df['Category_Prob'] == 'AGN'))

11

In [178]:
print('4FGL-DR2 number of sources:', 5064 + 723)
print(len(cat_df))
print(len(cat0_df))

4FGL-DR2 number of sources: 5787
5788
5788


In [179]:
for label in labels:
    n_other = np.sum((cat_df['Category_Prob'] == label) & other_mask)
    n_label = 1. * np.sum((cat_df['Category_Prob'] == label) & ass_mask)
    print('%s other: %i' % (label, n_other))
    if n_label:
        print('%s other / n assoc: %.2f' % (label, n_other / n_label))
    n_corr = 1. * n_other * n_unas / n_as
    res_dict['corrected'][label] = res_dict['uncorrected'][label] - n_corr
    print('%s uncorrected: %i' % (label, res_dict['uncorrected'][label]))
    print('%s corrected: %.1f' % (label, res_dict['corrected'][label]))
print('Assoc other:', sum(other_mask))
print('Unassoc other model:', (sum(other_mask) * n_unas / n_as))

AGN other: 70
AGN other / n assoc: 0.02
AGN uncorrected: 739
AGN corrected: 710.7
PSR other: 17
PSR other / n assoc: 0.09
PSR uncorrected: 64
PSR corrected: 57.1
OTHER other: 133
OTHER other / n assoc: 0.80
OTHER uncorrected: 274
OTHER corrected: 220.2
MIXED other: 127
MIXED other / n assoc: 0.33
MIXED uncorrected: 590
MIXED corrected: 538.6
Assoc other: 347
Unassoc other model: 140.36617325891774


In [180]:
# predictions based on probabilities
noc = 0.
for pop in ['AGN', 'PSR', 'OTHER'][:nclasses]:
    pred = []
    if nclasses == 2:
        pred_corr = []
    for alg in algs:
        label = '%s_%s' % (pop, alg)
        n0 = np.sum(cat_df.loc[unass_mask, label])
        pred.append(n0)
        if nclasses == 2:
            # correct for other    
            pop_other = np.sum(cat_df.loc[other_mask, label])
            n_corr = n0 - pop_other * n_unas / n_as
            pred_corr.append(n_corr)
    mean, delta_plus, delta_minus = aux.get_mean_dp_dm(pred)
    print('Number of %s: $%.1f^{+%.1f}_{-%.1f}$' % (pop, mean, delta_plus, delta_minus))
    if nclasses == 2:
        mean_corr, delta_plus, delta_minus = aux.get_mean_dp_dm(pred_corr)
        print('Number of %s (corr): $%.1f^{+%.1f}_{-%.1f}$' % (pop, mean_corr, delta_plus, delta_minus))
        noc += mean - mean_corr
if nclasses == 2:
    n_other = np.sum(other_mask) * n_unas / n_as
    print('n other model: %.1f' % n_other)
    print(noc)

Number of AGN: $948.0^{+122.7}_{-135.8}$
Number of PSR: $214.9^{+47.0}_{-41.8}$
Number of OTHER: $504.1^{+88.9}_{-102.6}$


In [181]:
#cat_df.columns

In [182]:
mv_mask = (cat_df['Missing_Values_Flag'] == 1)
np.sum(cat_df[mv_mask]['Category_%s' % cat] == 'UNAS')

9